In [ ]:
import pandas as pd
import numpy as np
from numpy import inf

In [ ]:
#Read search query file
sq = pd.read_csv('SearchQuery.csv', skiprows=1, usecols = ['Search term','Clicks', 'Impressions', 'Cost', 'Conversions'])
sq[0:5]

In [ ]:
#Segment each search term to a search word, and calculate the performance of a search word by adding up the performance of all the search terms that contain that search word
strst_terms=[]
strst_cost=[]
strst_impressions=[]
strst_clicks=[]
strst_conversions=[]
for i in range(len(sq)):
    terms=sq['Search term'][i].split()
    #print(terms)
    cost=sq['Cost'][i]
    impressions=sq['Impressions'][i]
    clicks=sq['Clicks'][i]
    conversions=sq['Conversions'][i]
    #print(cost)
    for t in range(len(terms)):
        strst_terms.append(terms[t])
        strst_cost.append(cost)
        strst_impressions.append(impressions)
        strst_clicks.append(clicks)
        strst_conversions.append(conversions)
#print(len(strst_terms))
#print(len(strst_cost))

print(strst_terms)



In [ ]:
#Create a new dataframe and show search words with aggregated cost, impression, clicks and conversion data
strst_list = pd.DataFrame(
    {'strst_terms': strst_terms,
     'strst_cost': strst_cost,
     'strst_impressions': strst_impressions,
     'strst_clicks': strst_clicks,
     'strst_conversions': strst_conversions,
     })
strst_list['strst_cost'] = strst_list['strst_cost'].str.replace(',', '').astype(float)
#strst_list['strst_clicks'] = strst_list['strst_clicks'].str.replace(',', '').astype(float)
strst_list['strst_conversions'] = strst_list['strst_conversions'].str.replace(',', '').astype(float)
strst_list.groupby('strst_terms')['strst_cost'].sum().sort_values(ascending=False)    

In [ ]:
#Calculate aggregated conversion rate, click through rate and cost per lead
out = strst_list.groupby(['strst_terms'], sort=0).apply(lambda g: pd.Series({
'strst_clicks': g.strst_clicks.sum(),
'strst_conversions': g.strst_conversions.sum(),
'strst_cost': g.strst_cost.sum(),
'strst_impressions': g.strst_impressions.sum(),
'count': g.strst_clicks.count(),
})   )
out['strst_CR'] = out['strst_conversions']/out['strst_clicks']
out['strst_CTR'] = out['strst_clicks']/out['strst_impressions']
out['strst_CPL'] = out['strst_cost']/out['strst_conversions']
out[ out['strst_CPL'] < inf].sort_values(by='strst_CPL', ascending = False)


In [ ]:
#Output a csv file of performance on search words and sort by cost
out = out.sort_values(by='strst_cost', ascending = False)
fresult=out.drop(out.index[0])
fresult[0:5]
fresult.to_csv('SQResult.csv')
